In [21]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import unicodedata


from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException


#soup.find() ne trouve que le premier tag dans l'html, pour tous les avoir, il faut utiliser soup.find_all()

#En utilisant les méthodes find et find all on peut spécifier le type de balise que l'on cherche ainsi que leur classe html

In [22]:
MY_LINKS = [
"https://www.nifc.gov/fire-information/nfn",
"https://www.nbcnews.com/western-wildfires",
"https://wildfiretoday.com/",
"https://www.hcn.org/search?SearchableText=wildfire",
"https://www.reddit.com/r/wildfires/",
"https://en.wikipedia.org/wiki/Wildfires_in_2020",
]

In [23]:
#WIKIPEDIA

#requests.get renvoie la réponse de la page, pour avoir le contenu, il faut utiliser text, ainsi on peut en faire une soup
html_text = requests.get(MY_LINKS[5]).text

soup = BeautifulSoup(html_text, 'lxml')

#getting div with "main" id
html_main_div = soup.find('div', {"id" : "mw-content-text"})

table = html_main_div.find_all('table')
df = pd.read_html(str(table))[3]

df




Coordination Center       Acres   Hectares  Suppression Costs  \
0         Alaska Interagency    171045.7    69219.7     $14,837,241.00   
1             Northwest Area   1925434.2   779195.6    $334,672,820.78   
2   Northern California Area   3961089.6  1602996.1  $1,369,875,556.25   
3   Southern California Area   1241246.5   502314.6    $751,084,644.00   
4           Northern Rockies    359948.6   145666.0     $71,770,047.00   
5                Great Basin    891689.5   360853.9    $236,649,112.00   
6             Southwest Area   1036287.6   419370.7    $192,069,000.96   
7        Rocky Mountain Area    818608.6   331279.1    $276,080,314.34   
8               Eastern Area     10508.4     4252.6        $522,398.58   
9              Southern Area   2678366.3  1083896.4     $14,692,891.11   
10                 Totals[a]  13094224.9  5299044.8  $3,262,254,026.02   

    Structures Destroyed  
0                      8  
1                   4473  
2                   7410  
3                   1824  
4                    222  
5                    172  
6                     63  
7                    212  
8                     19  
9                    313  
10                 14716

In [24]:
#NBC Western wildfires
html_text = requests.get(MY_LINKS[1]).text

soup = BeautifulSoup(html_text, 'lxml')

#getting div with "main" id
html_main_div = soup.find('div', class_ = "rail__container layout-leftRail layout-grid-container")

html_rows = html_main_div.find_all('div', class_ = "wide-tease-item__wrapper df flex-column flex-row-m flex-nowrap-m")

my_link_dict = {}
for r in html_rows:
    a_list = r.find_all('a')
    for a in a_list:
        if a.h2 != None:
            title = a.h2.text
            link = a['href']

            my_link_dict[title] = link

print(my_link_dict)

{"'We're one cigarette away': Illegal weed farms are a wildfire risk in parched California": 'https://www.nbcnews.com/news/us-news/fire-guns-poison-illegal-marijuana-farms-pose-deadly-risks-californias-rcna7153', 'On the front lines with hotshot teams fighting wildfires in the West': 'https://www.nbcnews.com/nightly-news/video/on-the-front-lines-with-hotshot-teams-fighting-wildfires-in-the-west-127268933832', 'Wildfire near Rocky Mountain National Park fully contained at 147 acres': 'https://www.nbcnews.com/news/us-news/wildfire-rocky-mountain-national-park-fully-contained-147-acres-rcna6258', 'Thousands of giant sequoias killed in California wildfires': 'https://www.nbcnews.com/science/science-news/thousands-giant-sequoias-killed-california-wildfires-rcna6182', 'Former California professor indicted on arson charges in 4 wildfires': 'https://www.nbcnews.com/news/us-news/former-california-professor-indicted-arson-charges-4-wildfires-rcna6099', 'Colorado pilot fighting wildfire was makin

In [25]:
for title in my_link_dict.keys():

    print()
    print('NEW_PAGE')
    print()

    print(title)
    link = my_link_dict[title]
    print(link)
    html_text = requests.get(link).text
    soup = BeautifulSoup(html_text, 'lxml')

    time = soup.find('time')
    if time != None:
        if time.has_attr('datetime'):

            print(time['datetime'])
            print()
    
    article = soup.find('article')
    paragraphs = article.find_all('p')
    for p in paragraphs:
        print(p.text)


NEW_PAGE

'We're one cigarette away': Illegal weed farms are a wildfire risk in parched California
https://www.nbcnews.com/news/us-news/fire-guns-poison-illegal-marijuana-farms-pose-deadly-risks-californias-rcna7153
Thu Dec 02 2021 01:40:21 GMT+0000 (UTC)

LAKE ELSINORE, Calif. — After a 2½-mile trek through thick brush, Mourad Gabriel stepped into a small clearing. A month earlier, this half-acre swath of the Cleveland National Forest, nearly invisible from the air, had been an illegal marijuana grow estimated to be worth more than $1.2 million. The U.S. Forest Service’s law enforcement officers had hacked down the plants, but Gabriel and his team were there to cart out nearly 3,000 pounds of trash and to clean up something else the drug traffickers left behind: poison. 
Gabriel, a regional wildlife ecologist for the Forest Service, spooned swabs of pesticide into a military-grade testing device to identify chemicals used by illicit farmers, which kill the forest’s wildlife. Recallin

In [26]:
#HCN
html_text = requests.get(MY_LINKS[3]).text

soup = BeautifulSoup(html_text, 'lxml')

html_rows = soup.find('div', class_ = "gsc_wrapper")

my_link_dict = {}
for r in html_rows:
    a_list = r.find_all('a')
    for a in a_list:
        if a.h2 != None:
            title = a.h2.text
            link = a['href']

            my_link_dict[title] = link

print(my_link_dict)


TypeError: 'NoneType' object is not iterable

In [27]:
#Scraping NBC news wild fires with Selenium and Beautifulsoup

PATH = "./chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get(MY_LINKS[1])

#accepting cookie policy
cookies = driver.find_element_by_id("cx_button_close")
cookies.click()

# finded = True is there is a load button
finded = True

while finded:
    try:
        search = driver.find_element_by_class_name("animated-ghost-button.animated-ghost-button--normal.feeds__load-more-button")
        search.click()
    except NoSuchElementException:
        finded = False

html_source = driver.page_source

driver.quit()

soup = BeautifulSoup(html_source, 'lxml')

#getting div with "main" id
html_main_div = soup.find('div', class_ = "rail__container layout-leftRail layout-grid-container")

html_rows = html_main_div.find_all('div', class_ = "wide-tease-item__wrapper df flex-column flex-row-m flex-nowrap-m")

my_link_dict = {}
for r in html_rows:
    a_list = r.find_all('a')
    for a in a_list:
        if a.h2 != None:
            title = a.h2.text
            link = a['href']

            my_link_dict[title] = link

data = []

with open("scarp_NBC.txt", "w", encoding="utf-8") as file: #this creates a .txt file that contains the data

    for title in my_link_dict.keys():
        d = {}
        
        file.write("\n")
        file.write('NEW_PAGE')
        file.write("\n")

        file.write(title + "\n") # Title
        d["Title"] = title
        link = my_link_dict[title] 
        file.write(link + "\n")  # Link
        d["Link"] = link
        html_text = requests.get(link).text
        soup = BeautifulSoup(html_text, 'lxml')

        time = soup.find('time')
        if time != None:
            if time.has_attr('datetime'):

                file.write(time['datetime'] + "\n") #Write datetime if exists
                d["Date"] = time['datetime']
                file.write("\n")
        
        article = soup.find('article')
        if article != None:
            paragraphs = article.find_all('p')
        big_p = ""
        for p in paragraphs:
            big_p = big_p + p.text +""
            file.write(p.text + "\n") # Write all paragraphs
        
        if big_p != "":
            d["Content"] = unicodedata.normalize("NFKD", big_p).rstrip()

        data.append(d)

nbc_df = pd.DataFrame(data)

C:\Users\SORLUC~1\AppData\Local\Temp/ipykernel_15672/3327526284.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
C:\Users\SORLUC~1\AppData\Local\Temp/ipykernel_15672/3327526284.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  cookies = driver.find_element_by_id("cx_button_close")
C:\Users\SORLUC~1\AppData\Local\Temp/ipykernel_15672/3327526284.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_class_name("animated-ghost-button.animated-ghost-button--normal.feeds__load-more-button")


In [28]:
print(nbc_df.isna().sum()) #Checking for missing values

Title       0
Link        0
Date       34
Content    35
dtype: int64


We have 34 missing date values in our data.

In [29]:
nbc_df

Title  \
0    'We're one cigarette away': Illegal weed farms...   
1    On the front lines with hotshot teams fighting...   
2    Wildfire near Rocky Mountain National Park ful...   
3    Thousands of giant sequoias killed in Californ...   
4    Former California professor indicted on arson ...   
..                                                 ...   
143  Over 900,000 acres scorched by fires in Oregon...   
144  Power outages hamper evacuation warnings and d...   
145  7 deaths confirmed as wildfire rages in Northe...   
146  Wildfire north of Sacramento is now largest in...   
147          Deadly wildfires sweep through West Coast   

                                                  Link  \
0    https://www.nbcnews.com/news/us-news/fire-guns...   
1    https://www.nbcnews.com/nightly-news/video/on-...   
2    https://www.nbcnews.com/news/us-news/wildfire-...   
3    https://www.nbcnews.com/science/science-news/t...   
4    https://www.nbcnews.com/news/us-news/former-ca...   
..                                                 ...   
143  https://www.msnbc.com/mtp-daily/watch/over-900...   
144  https://www.nbcnews.com/business/business-news...   
145  https://www.nbcnews.com/news/us-news/7-bodies-...   
146  https://www.nbcnews.com/news/us-news/wildfire-...   
147  https://www.nbcnews.com/nightly-news/video/dea...   

                                        Date  \
0    Thu Dec 02 2021 01:40:21 GMT+0000 (UTC)   
1                                        NaN   
2    Sun Nov 21 2021 03:09:39 GMT+0000 (UTC)   
3    Fri Nov 19 2021 20:00:52 GMT+0000 (UTC)   
4    Fri Nov 19 2021 11:57:32 GMT+0000 (UTC)   
..                                       ...   
143                                      NaN   
144  Fri Sep 11 2020 17:37:16 GMT+0000 (UTC)   
145  Fri Sep 11 2020 05:30:00 GMT+0000 (UTC)   
146  Fri Sep 11 2020 01:23:00 GMT+0000 (UTC)   
147                                      NaN   

                                               Content  
0    LAKE ELSINORE, Calif. — After a 21⁄2-mile trek...  
1                                                  NaN  
2    A wildfire burning for four days in steep terr...  
3    LOS ANGELES — Lightning-sparked wildfires kill...  
4    A former California professor accused of setti...  
..                                                 ...  
143                                                NaN  
144  Danielle Mays lives in a wood cabin at the end...  
145  Seven more deaths have been confirmed where a ...  
146  California has never before seen a fire as lar...  
147                                                NaN  

[148 rows x 4 columns]

In [20]:
#now exporting data to CSV
nbc_df.to_csv("nbc_df.csv", encoding="utf-8")